In [1]:
import pandas as pd
import requests
import sys
from bs4 import BeautifulSoup
import multiprocessing as mp
from tqdm import tqdm
from datetime import datetime
import re
from proxycrawl import CrawlingAPI, ScraperAPI, LeadsAPI

In [2]:
base_folder = 'Butterfly Past Low Priority June'
api = CrawlingAPI({ 'token': 'ubxeiSnB52uiJVjGDWSa0g&url=https%3A%2F%2Fwww.amazon.com%2Fdp%2FB00JITDVD2' })

In [3]:
def get_search(search):
    # Initial query
    
    search_term = '+'.join(search.split(' '))
    URL = 'https://www.ebay.com/sch/i.html?_sacat=0&LH_Sold=1&_udlo=&_udhi=&_samilow=&_samihi=&_sop=12&_dmd=1&_ipg=200&LH_Complete=1&_fosrp=1&_nkw=%22'+search_term+'%22&rt=nc&LH_PrefLoc=2'
    page = api.get(URL,{'country':'US'})
    soup = BeautifulSoup(page['body'], 'html.parser')
    
    found = False
    cnt = 0
    while not found:
        if cnt > 10:
            return URL, 0, False, cnt
        cnt = cnt+1
        
        page = api.get(URL,{'country':'US'})
        soup = BeautifulSoup(page['body'], 'html.parser')
        try:
            soup.find('div', id='cbelm').find_all('span', class_='rcnt')[0].text
            found = True
        except:
            found = False
        
    
    # Get number of results
    try:
        count = soup.find('div', id='cbelm').find_all('span', class_='rcnt')[0].text
    except:
        try:
            count = soup.find_all('span', class_='nllclt')[0].find_all('b')[0].text
        except:
            print(URL)
            
            count = soup.find_all('span', class_='rcnt')[0].text
        
    #If no results are returned then skip it
    if count == '0':
        return URL, count, False, cnt

    # Get the main list container of the results
    results = soup.find('ul', id='ListViewInner')

    # Get listing items
    items = results.find_all('li', class_='sresult', recursive=False)

    # Capture all data in this list
    dict_list = []

    # Loop through listings
    for item in items:
        # Define empty dictionary
        dict_item = {
            'id': None,
            'title': None,
            'link': None,
            'image': None,
            'price_sold': None,
            'end_date': None,
            'origin': None,
            'seller_name': None,
            'seller_link': None,
            'sell_type': None,
            'num_bids': 0,
            'num_sold': 1,
            #'description': None
        }

        # Grab all the high level data data
        id = item.get('id')
        title_and_link = item.find_all('a', class_='vip')[0]
        title = title_and_link.get('title').replace('Click this link to access ','')
        link = title_and_link.get('href')
        price_info = item.find_all('ul', class_='lvprices', recursive=False)[0].find_all('li', class_='lvprice')[0]
        price_best_offer = price_info.find_all('span', class_='sboffer')
        price_bid_sold = price_info.find_all('span', class_='bidsold')
        end_date = item.find_all('ul', class_='lvdetails', recursive=False)[0].find_all('span', class_='tme')[0].find_all('span')[0].text.strip()
        
        try:
            image = item.find_all('img', class_='img')[0].get('src')
        except:
            image = 'NA'


        # Description
        #description_src = deeper_info.find_all('iframe')[0].get('src')
        #description = ' '.join(BeautifulSoup(requests.get(description_src).content, 'html.parser').find('div', id='ds_div').text.strip().split())

        # Put the data into the dictionary and append to the list
        dict_item['id'] = id
        dict_item['title'] = title
        dict_item['link'] = link
        dict_item['image'] = image
        if len(price_best_offer) > 0:
            dict_item['price_sold'] = price_best_offer[0].text.strip()
        else:
            dict_item['price_sold'] = price_bid_sold[0].text.strip()
        dict_item['end_date'] = end_date

        dict_list.append(dict_item)

    # convert data to pandas dataframe and save species search result to CSV
    df = pd.DataFrame(dict_list)
    df.to_csv(base_folder+"/past sales/past_sales_"+search+".csv", index=False, encoding = 'utf-8-sig')
    
    return URL, count, True, cnt

In [ ]:
# Get today's date
date = '2022-06-01'
part = 1

# Load file containing past scraps so that we can resume where we left off if we stopped
try:
    done_list = pd.read_csv(base_folder+'/past_search_result_'+str(part)+'_'+date+'.csv')
    done = done_list['species'].tolist()
    done_list = done_list.to_dict('records')
except:
    done_list = []
    done = []

# Get list of species
search_list = pd.read_csv(base_folder+'/low_priority_butterfly_list.csv')['Species'].tolist()
search_list = search_list[(5000*(part-1)):(5000*part)]

# Loop through list of species, making sure to skip the ones we already finished
for search in tqdm(search_list):
    if search in done:
        continue
    # Get scrape results
    URL, count, result, captcha_cnt = get_search(search)
    
    # Append scrape results to the list
    search_result_dict = {'species': None, 'found': None, 'URL': None, 'count': None, 'captcha_count':None}
    search_result_dict['URL'] = URL
    search_result_dict['species'] = search
    search_result_dict['found'] = result
    search_result_dict['count'] = count
    search_result_dict['captcha_count'] = captcha_cnt
    done_list.append(search_result_dict)
    
    # Save metadata
    df_result = pd.DataFrame(done_list)
    df_result.to_csv(base_folder+"/past_search_result_"+str(part)+'_'+date+".csv", index=False)
    

  1%|          | 27/5000 [05:12<15:30:21, 11.22s/it]